Hanna Olsson 
Johan Östling


In [ ]:
!pip install tensorflow

In [ ]:
!pip install keras

In [ ]:
!pip install Pillow

# Imports

In [12]:
import keras
from PIL import Image, ImageDraw, ImageFont


**Mount to drive**

In [23]:
from google.colab import drive

drive.mount('/content/drive')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# Introduction

In [26]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

data_gen = ImageDataGenerator(rescale=1.0/255)
#?usp=share_link
imgdir = '/content/drive/MyDrive/1KjQ-TBLBsRQ21vLtSZ7SYGKnGtbI0IKC' # or wherever you put them...
img_size = 64
batch_size = 32

train_generator = data_gen.flow_from_directory(
        imgdir + '/train',
        target_size=(img_size, img_size),
        batch_size=batch_size,
        class_mode='binary',
        classes=['other', 'car'],
        seed=12345,
        shuffle=True)

Found 0 images belonging to 2 classes.


In [21]:
Xbatch, Ybatch = train_generator.next()

# Part 1: Training a convolutional neural network

# Part 2: Data augmentation

**Interlude: Applying a pre-trained convolutional neural network**

# Part 3: Using VGG-16 as a feature extractor

# Part 4: Visualizing the learned features